# Assignment 3

### <span style="color:chocolate"> Submission requirements </span>

Your work will not be graded if your notebook doesn't include output. In other words, <span style="color:red"> make sure to rerun your notebook before submitting to Gradescope </span> (Note: if you are using Google Colab: go to Edit > Notebook Settings  and uncheck Omit code cell output when saving this notebook, otherwise the output is not printed).

Additional points may be deducted if these requirements are not met:

    
* Comment your code;
* Each graph should have a title, labels for each axis, and (if needed) a legend. Each graph should be understandable on its own;
* Try and minimize the use of the global namespace (meaning, keep things inside functions).
---

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt
import keras_tuner as kt
from keras_tuner import HyperParameters

This lab continues our study of linear regression. You'll train your first models with Tensorflow, using a real dataset to predict car prices from their features. Note that Tensorflow is a rapidly changing library. This means you'll often see warnings about deprecations. You can ignore the warnings in our labs.

---
### Step 1: Data ingestion

You'll use the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile)  from 1985 Ward's Automotive Yearbook that is part of the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets).

In [ ]:
# Provide the names for the feature columns since the CSV file with the data
# does not have a header row.
cols = ['losses', 'make', 'fuel-type', 'aspiration', 'num-doors',
        'body-style', 'drive-wheels', 'engine-location', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-type', 'num-cylinders',
        'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio',
        'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price','symboling']

# Load the data from a CSV file into a pandas dataframe. Remember that each row
# is an example and each column in a feature.
car_data_init = pd.read_csv(
    'https://github.com/aagarwal17/datasci-207-arun/blob/main/assignments/cars_data.csv?raw=true',
    sep=',', names=cols, skiprows = 1, encoding='latin-1')

# Display top five rows
print('Shape of data:', car_data_init.shape)
car_data_init.head()

---
### Step 2: Data preprocessing

This step is essential for preparing the data in a format that is suitable for ML algorithms. It helps ensure data quality and improvements in model performance.

### <span style="color:chocolate">Exercise 1:</span> Column selection (5 points)

To keep things simple, you will:

1. Retain only the following columns: ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']. Name the new dataframe *car_data*.
2. Display the data type of each column;
3. Convert the data type of each columns to numeric. Coerce missing values to NaN. Hint: use <span style="color:chocolate">pd.to_numeric()</span> method;
4. Display the data type of each column after the transformation performed at point 3.


In [ ]:
# YOUR CODE HERE

# 1.
car_data = car_data_init[['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']]

# 2
print("Data types of each column:")
print(car_data.dtypes)

# 3.
for column in car_data.columns:
    car_data[column] = pd.to_numeric(car_data[column], errors='coerce')

# 4.
print("Data types after conversion:")
print(car_data.dtypes)


### <span style="color:chocolate">Exercise 2:</span> Example (row) selection (5 points)

To keep things simple again, you will:

1. Print the shape of the car_data;

2. Remove examples (rows) that have missing value(s). Note that in doing so, you will overwrite the car_data dataset. You should end up with 199 examples after this cleaning.

3. Print the shape of the car_data again.

It's important to acknowledge that there are multiple approaches to handling missing features, and simply discarding examples with any missing feature, though straightforward, may not be the most optimal solution. However, for the sake of simplicity, you will implement this strategy in this assignment.

In [ ]:
# YOUR CODE HERE
# 1.
print("Shape of car_data before cleaning:", car_data.shape)

# 2.
car_data = car_data.dropna()

# 3.
print("Shape of car_data after cleaning:", car_data.shape)


### <span style="color:chocolate">Exercise 3:</span> Data shuffling (10 points)

Since you'll be using Batch Gradient Descent (BGD) for training, it is important that **each batch is a random sample of the data** so that the gradient computed is representative. Note that the original data (above) appears sorted by *make* in alphabetic order.

Using NumPy and Pandas methods:

1. Create a list of indices corresponding to the rows in the car_data dataset. Call this list *indices*. Print this list;

2. Shuffle *indices* using the <span style="color:chocolate">np.random.permutation()</span> method. Call the resulting array *shuffled_indices*. Print this array;
    
3. Use the method <span style="color:chocolate">dataframe.reindex()</span> to change the ordering of the car_data dataset based on the order in the *shuffled_indices* array. Note that in doing so, you will overwrite the original dataset. Print the top 5 rows.

In [ ]:
np.random.seed(0)
# 1.
indices = list(car_data.index)
print("Original indices:", indices)

# 2.
shuffled_indices = np.random.permutation(indices)
print("Shuffled indices:", shuffled_indices)

# 3.
car_data = car_data.reindex(shuffled_indices)
print("Top 5 rows after shuffling:")
print(car_data.head())


### <span style="color:chocolate">Exercise 4:</span> Define outcome and features (5 points)

Create two dataframes as follows:

1. The first dataframe contains our outcome of interest: ['price']. Note, this is what we are aiming to predict. Name this dataframe Y. Print shape of Y.
2. The second dataframe contains our features of interest: ['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg']. Name this dataframe X. Print shape of X.


In [ ]:
# YOUR CODE HERE
# 1.
Y = car_data[['price']]
print("Shape of Y:", Y.shape)

# 2.
X = car_data[['horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg']]
print("Shape of X:", X.shape)

### <span style="color:chocolate">Exercise 5:</span> Data splits (10 points)

Using the <span style="color:chocolate">train_test_split()</span> method available in scikit-learn:
1. Partition the (X, Y) data into training, validation, and test sets using a splitting rule of [60%, 20%, 20%], with a random state set to 1234. Name the resulting dataframes as follows: X_train, X_val, X_test, Y_train, Y_val, Y_test. Hint: To create these three partitions you will utilize the train_test_split() method twice (all the other arguments of the method are set to default values.). You should obtain [119, 40, 40] examples for training, validation, and test, respectively.
2. Print the shape of each dataframe.

Note: The validation set is crucial for evaluating different hyperparameter configurations and selecting those that yield optimal model performance. This approach avoids utilizing the test dataset during model training, as it is assumed to be "unknown" at that stage.

In [ ]:
# YOUR CODE HERE

# 1.
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=1234)

# 2.
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=1234)

# 3.
print("Shapes of datasets:")
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("X_test:", X_test.shape)
print("Y_train:", Y_train.shape)
print("Y_val:", Y_val.shape)
print("Y_test:", Y_test.shape)


### <span style="color:chocolate">Exercise 6:</span> Data standardization (10 points)

With this concept in mind, complete the following tasks:

1. Output the quantile values (0.25, 0.5, 0.75, 0.95) for all features in the X_train dataset. Are these values uniformly scaled across features?

2. Standardize all features in X_train, X_val, and X_test. Label the resulting dataframes as X_train_std, X_val_std, and X_test_std, respectively. Hint: standardize the validation and test data using the mean and standard deviation computed from the training data. Why?

3. Similar to point 2. but now standardize the outcome variable. Label the resulting dataframes as Y_train_std, Y_val_std, and Y_test_std.

In [ ]:
# YOUR CODE HERE

# 1.
quantiles = X_train.quantile([0.25, 0.5, 0.75, 0.95])
print("Quantile values for X_train:")
print(quantiles)
print("Q1: By examining the quantiles, we see the values are not uniformly scaled across features")

# 2.
X_mean = X_train.mean()
X_std = X_train.std()

X_train_std = (X_train - X_mean) / X_std
X_val_std = (X_val - X_mean) / X_std
X_test_std = (X_test - X_mean) / X_std
print("Q2: We standardize the validation and test data using the mean and standard deviation computed from the training data to prevent data leakage. That is, we want to keep the validation and test sets unseen by the model during training to avoid overfitting.")

# 3.
Y_mean = Y_train.mean()
Y_std = Y_train.std()

Y_train_std = (Y_train - Y_mean) / Y_std
Y_val_std = (Y_val - Y_mean) / Y_std
Y_test_std = (Y_test - Y_mean) / Y_std
print("We standardize the outcome variable so that the model puts the target variable on the same scale as the features, preventing bias.")


---
### Step 3: Exploratory data analysis (EDA)

EDA plays a very important role in ML. The goal here is to develop a good understanding of our training dataset, identify any data quality issues, understand patterns and relationships, which in turn, aids in subsequent modeling and interpretations.

### <span style="color:chocolate">Exercise 7:</span> Scatterplot matrix (10 points)

In this exercise you will use some simple yet useful techniques to visualize the distribution of the data.

Let's start with:

1. A scatterplot matrix to visualize the pair-wise correlations between different features and outcome in the (X_train_std, Y_train_std) data. You will use the <span style="color:chocolate">sns.pairplot()</span> method from the seaborn library imported at the top of the notebook;
2. Is any of the variables in the data normally distributed? Is it necessary for the explanatory or target variable to be normally distributed in order to train a ML model?

In [ ]:
# YOUR CODE HERE

# 1.
data = pd.concat([X_train_std, Y_train_std], axis=1)
sns.pairplot(data)
plt.show()

# 2.
print("Q2a. We determine if any of the variables in the data are normally distributed by looking at the plots along the diagonal. \nWe see that peak_rpm and highway-mpg look semi-normal. \nOtherwise, the other features are non-normally distributed (do not show a bell-shaped, symmetric distribution).")
print("Q2b. No, it is not necessary for the explanatory or target variable variables to be normally distributed to train a ML model. \nWhile some models like linear regression may perform better when the variables are normally distributed, the explanatory or target variable does not need to be normally distributed for this. \nWe state in the module demo that 'ML prediction does not require the outcome and feature variables to be normally distributed! \nThis is a requirement for inference and hypothesis testing but not for predictive analysis'")

### <span style="color:chocolate">Exercise 8:</span> Correlation matrix (10 points)

In this exercise you will:

1. Plot a correlation matrix in the form of a heatmap to visualize the linear relationships between different features and outcome in the (X_train_std, Y_train_std) data. Hint: this example here is very useful: https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    
2. Answer the following questions:
 - Which two features are likely to be most redundant?
 - Which feature is likely to be least useful for predicting price?

In [ ]:
# YOUR CODE HERE
# 1.
correlation_matrix = pd.concat([X_train_std, Y_train_std], axis=1).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', cbar=True)
plt.title("Correlation Matrix Heatmap")
plt.show()

# 2.
redundant_features = correlation_matrix.abs().unstack().sort_values(ascending=False)
redundant_pairs = redundant_features[(redundant_features < 1) & (redundant_features > 0.8) | (redundant_features > -1) & (redundant_features < -0.8)]
print(f"Q2a. The most redundant features are those with a correlation close to 1 or -1 (aka highly correlated). \nBy my calculation above or by examining the heatmap, we see the top pairs/most redundant features are\n: {redundant_pairs.head(4)} \n(Note: a heatmap includes every value twice unless I put a mask on it to just show the needed triangle, so the top 4 includes each pairing twice.)")

least_useful_feature = correlation_matrix['price'].abs().idxmin()
print(f"Q2b. The feature with the lowest correlation to the target variable (price) is likely to be least useful. \nBy examining the heatmap or using our calculation below, we find the least useful feature to be: {least_useful_feature}")

---
### Step 4: Modeling

### <span style="color:chocolate">Exercise 9:</span> Baseline model (5 points)

Let's start by evaluating a baseline model. Precisely, you'll use the average price of cars in the training set as our baseline model -- that is, the baseline always predicts the average price regardless of the input.

1. Implement this baseline using the Y_train_std data and print the average price. Note: You can revert the price variable to the original scale for interpretation purposes.

In [ ]:
# YOUR CODE HERE
# 1.
mean_price_std = Y_train_std.mean()
mean_price = (mean_price_std * Y_std) + Y_mean
print(f"Average price in the training set (original scale): ${mean_price.values}")


### <span style="color:chocolate">Exercise 10:</span> Improvement over Baseline with TensorFlow (10 points)

Let's train a linear regression model much like we did in the previous assignment, but this time using TensorFlow.

1. Fill in the <span style="color:green">NotImplemented</span> parts of the build_model() function below by following the instructions provided as comments. Hint: refer to Demo 3 in [bCourses/Modules/Live Session Demos](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) for an example.
2. Build and compile a model using the build_model() function and the (X_train_std, Y_train_std) data. Set learning_rate = 0.0001. Call the resulting object *model_tf*.
3. Train *model_tf* using the (X_train_std, Y_train_std) data. Set num_epochs = 5. Pass the (X_val_std, Y_val_std) data for validation. Hint: see the documentation behind the [tf.keras.Model.fit()](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) method.
3. Generate a plot with the loss values on the y-axis and the epoch number on the x-axis for visualization. Make sure to include axes name and title. Hint: check what the [tf.keras.Model.fit()](https://bcourses.berkeley.edu/courses/1534588/files/88733489?module_item_id=17073646) method returns.

More notes on point 1: the idea is to build a *computational graph* for linear regression, and then send data through it. There are many ways to build graphs, but [TenforFlow Keras API](https://www.tensorflow.org/api_docs/python/tf/keras) is recommended.

In [ ]:
def build_model(num_features, learning_rate):
  """Build a TF linear regression model using Keras.

  Args:
    num_features: The number of input features.
    learning_rate: The desired learning rate for SGD.

  Returns:
    model: A tf.keras model (graph).
  """
  # This is not strictly necessary, but each time you build a model, TF adds
  # new nodes (rather than overwriting), so the colab session can end up
  # storing lots of copies of the graph when you only care about the most
  # recent. Also, as there is some randomness built into training with SGD,
  # setting a random seed ensures that results are the same on each identical
  # training run.
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)

  # Build a model using keras.Sequential. While this is intended for neural
  # networks (which may have multiple layers), we want just a single layer for
  # linear regression.
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(
      units=1,        # output dim
      input_shape=(num_features,),  # input dim
      use_bias=True,               # use a bias (intercept) param
      kernel_initializer=tf.ones_initializer,  # initialize params to 1
      bias_initializer=tf.ones_initializer,    # initialize bias to 1
  ))

  # We need to choose an optimizer. We'll use GD, which is actually mini-batch GD
  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

  # Finally, compile the model. This finalizes the graph for training.
  # We specify the MSE loss and the optimizer above
  model.compile(
        optimizer=optimizer,
        loss='mse',
    )

  return model


In [ ]:
tf.random.set_seed(0)
# 2. Build and compile model
# YOUR CODE HERE
learning_rate = 0.0001
model_tf = build_model(X_train_std.shape[1], learning_rate)

# 3. Fit the model
# YOUR CODE HERE
history = model_tf.fit(X_train_std, Y_train_std, epochs=5, validation_data=(X_val_std, Y_val_std))

# 4.
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.show()


---
### Step 5: Hyperparameter tuning

Hyperparameter tuning is a crucial step in optimizing ML models. It involves systematically adjusting hyperparameters such as learning rate, number of epochs, and optimizer to find the model configuration that leads to the best generalization performance.

This tuning process is typically conducted by monitoring the model's performance on the validation vs. training set. It's important to note that using the test set for hyperparameter tuning can compromise the integrity of the evaluation process by violating the assumption of "blindness" of the test data.

### <span style="color:chocolate">Exercise 11:</span> Hyperparameter tuning (10 points)

1. Fine-tune the **learning rate** and **number of epochs** hyperparameters of *model_tf* to determine the setup that yields the most optimal generalization performance. Feel free to explore various values for these hyperparameters. Hint: you can manually test different hyperparameter values or you can use the [Keras Tuner](https://www.tensorflow.org/tutorials/keras/keras_tuner). If you decide to work with the Keras Tuner, define a new model building function named <span style="color:chocolate">build_model_tuner()</span>.

After identifying your preferred model configuration, print the following information:

2. The learned parameters of the tuned model (this should include the bias term). Hint: use  <span style="color:chocolate">[model_name].layers[0].get_weights()</span>.
3. The loss at the final epoch on both the training and validation datasets;
4. The difference between the last-epoch loss observed on the training and validation datasets.


Please note that we will consider 'optimal model configuration' any last-epoch training loss that is below 0.31 and any last epoch validation loss that is below 0.48.

In [ ]:
# pip install -q -U keras-tuner

In [ ]:
tf.random.set_seed(0)
# YOUR CODE HERE
from keras_tuner import HyperModel, Hyperband

# 1. Defining the model for tuning using Keras Tuner
def build_model_tuner(hp):
    """Build a TF linear regression model using Keras for hyperparameter tuning.

    Args:
        num_features: The number of input features.
        learning_rate: The desired learning rate for SGD.

    Returns:
        model: A tf.keras model (graph).
    """

    tf.keras.backend.clear_session()
    tf.random.set_seed(0)

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(
        units=1,
        input_shape=(X_train_std.shape[1],),
        use_bias=True,
        kernel_initializer=tf.ones_initializer,
        bias_initializer=tf.ones_initializer,
    ))

    # Hyperparameters for tuning
    learning_rate = hp.Choice('learning_rate', values=[1e-1,1e-2, 5e-2, 1e-3, 5e-3, 1e-4, 1e-5])
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='mse')
    return model

# Set up the hyperparameter tuning using Keras Tuner
tuner = Hyperband(
    build_model_tuner,
    objective='val_loss',
    max_epochs=10,
    factor=3,
    directory='hyperparameter_tuning',
    project_name='linear_regression_tuning'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Perform hyperparameter search
tuner.search(X_train_std, Y_train_std, epochs=50, validation_data=(X_val_std, Y_val_std), callbacks=[stop_early], verbose=0)

# Getting the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Printing the best hyperparameters
print("Best hyperparameters:", best_hyperparameters.values)

# Finding Ideal number of epochs
best_model = tuner.hypermodel.build(best_hyperparameters)
history = best_model.fit(X_train_std, Y_train_std, epochs=50, validation_data=(X_val_std, Y_val_std), verbose=1)
val_loss_per_epoch = history.history['val_loss']
best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

# Retraining model with best hyperparams:
best_model = tuner.hypermodel.build(best_hyperparameters)
history = best_model.fit(X_train_std, Y_train_std, epochs=best_epoch, validation_data=(X_val_std, Y_val_std), verbose=1)

# 2. Getting learned weights and bias from the best model
weights, bias = best_model.layers[0].get_weights()
print("Learned Parameters (Weights):", weights)
print("Learned Bias Term:", bias)

# 3. Printing the loss at the final epoch for both training and validation datasets
final_train_loss = best_model.history.history['loss'][-1]
final_val_loss = best_model.history.history['val_loss'][-1]
print(f"Final Training Loss: {final_train_loss}")
print(f"Final Validation Loss: {final_val_loss}")

# 4. Printing the difference between the last-epoch training and validation losses
loss_difference = final_train_loss - final_val_loss
print(f"Loss Difference (Train - Validation): {loss_difference}")

# Plotting the training and validation loss curves
plt.plot(best_model.history.history['loss'], label='Training Loss')
plt.plot(best_model.history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

---
### Step 6: Evaluation and Generalization


Now that you've determined the optimal set of hyperparameters, it's time to evaluate your optimized (tuned) model on the test data to gauge its performance in real-world scenarios, commonly known as inference.

### <span style="color:chocolate">Exercise 12:</span> Computing MSE (10 points)

1. Calculate the MSE on both (X_train_std, Y_train_std) and (X_test_std, Y_test_std) datasets. Hint: You can utilize the <span style="color:chocolate">model.evaluate()</span> method provided by tf.keras.

2. Does the model demonstrate strong generalization capabilities? Provide an explanation based on your observations.

4. Generate a plot to visualize the accuracy of the predictions. Plot the actual (observed) Y_test values on the x-axis and the predicted Y_test values on the y-axis. Additionally, include a 45-degree line in the plot for reference. Ensure that the plot contains appropriate axis labels and a title. Provide commentary on the model's fit based on this visualization. Hint: You can utilize the <span style="color:chocolate">model.predict()</span> method available in tf.keras.

In [ ]:
# YOUR CODE HERE

# 1.
train_mse = best_model.evaluate(X_train_std, Y_train_std, verbose=0)
test_mse = best_model.evaluate(X_test_std, Y_test_std, verbose=0)
print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

# 2.
if test_mse <= train_mse:
    print("The model demonstrates strong generalization capabilities as the test MSE is lower than or equal to the training MSE (or at least relatively similar to the train MSE).")
else:
    print("The model may be overfitting/does not demonstrate strong generalization capabilities because the test MSE is higher than the training MSE.")

# 3.
y_pred = best_model.predict(X_test_std)

f, ax = plt.subplots(figsize=(8, 6))
ax.scatter(Y_test_std, y_pred, color='blue', alpha=0.6)
ax.axline([ax.get_xlim()[0], ax.get_ylim()[0]], [ax.get_xlim()[1], ax.get_ylim()[1]], color='red', linestyle='--', linewidth=1)
plt.xlabel('Actual Price (Y_test)')
plt.ylabel('Predicted Price (Y_pred)')
plt.title('Actual vs Predicted Price (Test Set)')
plt.grid(True)
plt.show()

print("Q3. Since the points do not seem to lie on or near the 45-degree line (red dashed line), the model may have not made good predictions. \nA perfect model would have all the points exactly on the line, but our model seems to have many points above and far from the line.")
print("This suggests that the model is overestimating the prices for many of the cars in the test set.\nAlthough the model's predictions are not perfect, they are still generally close to the true values, indicating that it has captured some useful patterns, but there is room for further improvement in accuracy.")

----
#### <span style="color:chocolate">Additional practice question</span> (not graded)

In Exercise 12, you reported an aggregated MSE. Let's revisit the exercise by:

1. Performing a subgroup evaluation of the model. Specifically, calculate the test data MSE for the following makes: ['alfa-romero', 'audi', 'chevrolet', 'dodge', 'honda'].
2. Addressing the question: Is the model "fair" across each make?

In [ ]:
# YOUR CODE HERE
from sklearn.metrics import mean_squared_error

# 1.
# makes_of_interest = ['alfa-romero', 'audi', 'chevrolet', 'dodge', 'honda']
# test_data_filtered = X_test_std[Y_test_std.index.isin(makes_of_interest)]
# y_pred_filtered = best_model.predict(test_data_filtered)
# y_pred_filtered_original = (y_pred_filtered * Y_train_std.std()) + Y_train_std.mean()
# y_test_filtered_original = (Y_test_std[Y_test_std.index.isin(makes_of_interest)] * Y_train_std.std()) + Y_train_std.mean()
# mse_per_make = {}
# for make in makes_of_interest:
#     make_indices = test_data_filtered[test_data_filtered['make'] == make].index
#     y_pred_make = y_pred_filtered_original[make_indices]
#     y_test_make = y_test_filtered_original[make_indices]

#     mse_per_make[make] = mean_squared_error(y_test_make, y_pred_make)
# print("MSE for each make:")
# for make, mse in mse_per_make.items():
#     print(f"{make}: {mse}")